In [7]:
import os
import json

# Define the directory where your .txt files are located
input_directory = "/home/abrham/Desktop/10x/week - 7/LLM-Finetuning_Generate-Amharic-based-creative-text-Ad-contents/notebooks"

# Define the directory where you want to save the .jsonl files
output_directory = "/home/abrham/Desktop/10x/week - 7/LLM-Finetuning_Generate-Amharic-based-creative-text-Ad-contents/notebooks"

# Ensure the output directory exists, if not create it
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Iterate through each .txt file in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith("train_dataset.txt"):
        with open(os.path.join(input_directory, filename), "r", encoding="utf-8") as f:
            # Read the contents of the text file
            lines = f.readlines()
            for line in lines:
                # Construct the JSON object
                data = {
                    "input": line.strip()
                }

                # Define the output file path
                output_filename = os.path.splitext(filename)[0] + ".jsonl"
                output_path = os.path.join(output_directory, output_filename)

                # Write the JSON object to the .jsonl file
                with open(output_path, "a", encoding="utf-8") as json_file:
                    json.dump(data, json_file, ensure_ascii=False)
                    json_file.write("\n")  # Add newline to separate entries in .jsonl file


In [12]:
import sentencepiece as spm
from datasets import load_dataset

# Loading the dataset
train_dataset = load_dataset('json', data_files='train_dataset.jsonl', split='train')
eval_dataset = load_dataset('json', data_files='eval_dataset.jsonl', split='train')

# Load the trained SentencePiece model
sp = spm.SentencePieceProcessor()
sp.Load('mt.model') # Replace with the path to your trained SentencePiece model

def formatting_func(example):
    return example['input']

def generate_and_tokenize_prompt(example):
    formatted_text = formatting_func(example)
    tokenized_ids = sp.EncodeAsIds(formatted_text)
    return {"tokenized_input": tokenized_ids}

# Apply the tokenization function to the datasets
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

Map: 100%|██████████| 4001/4001 [00:01<00:00, 2555.44 examples/s]


In [13]:
tokenized_train_dataset

Dataset({
    features: ['input', 'tokenized_input'],
    num_rows: 41907
})

In [14]:
# Iterate over the first few examples of the tokenized validation dataset
for example in tokenized_val_dataset.select(range(5)):
    print("Tokenized Input:", example["tokenized_input"])


Tokenized Input: [477, 298, 812, 440, 605, 5, 150, 493, 18, 3, 532, 449, 6, 41, 6, 15, 708, 217, 1040, 413, 3, 498, 55, 477, 307, 291, 298, 812, 440, 605, 5, 150, 493, 18, 3, 532, 449, 6, 41, 6, 15, 708, 217, 4, 1040, 413, 1830, 34, 32, 6, 41, 59, 5, 135, 20, 5, 135, 4, 269, 88, 383, 1013, 31, 63, 1040, 223, 239, 57, 11, 826, 1633, 117, 8, 924, 374, 687, 179, 694, 588, 568]
Tokenized Input: [209, 223, 951, 1401, 23, 3, 1857, 3, 97, 18, 10, 129, 19, 266, 27, 1366, 3, 59, 11, 32, 42, 272, 81, 27, 30, 373, 483, 209, 223, 951, 1401, 8, 83, 28, 344, 3, 252, 105, 508, 26, 4, 6, 3, 21, 210, 252, 3, 1007, 252, 495, 11, 216, 61, 4, 4, 16, 37, 106, 282, 8, 719, 269, 88, 23, 3, 865, 10, 162, 1111, 5, 6, 114, 21, 6, 8, 243, 465, 38, 124, 1564, 14, 611, 223, 408, 57, 321, 87, 441, 388, 233, 972, 33, 159]
Tokenized Input: [90, 3, 88, 29, 12, 13, 1175, 40, 728, 123, 861, 814, 553, 21, 310, 55, 5, 196, 303, 3, 981, 375, 108, 90, 3, 88, 29, 12, 13, 1175, 40, 4, 728, 123, 861, 1088, 3, 565, 29, 36, 1193

In [15]:
total_tokens = sum(len(example["tokenized_input"]) for example in tokenized_val_dataset)
print("Total Number of Tokens:", total_tokens)

Total Number of Tokens: 538249
